#**📩   Export Deep Crawl Reports to Excel**

In [ ]:
!pip install deepcrawl
import deepcrawl, pandas as pd
api_key = "API_KEY"
api_id = "API_ID"
account_id = "ACCOUNT_ID" # Find this in URL in interface
project_search_string = "PROJECT_NAME" # Guess at the name of the project

In [ ]:
dc = deepcrawl.DeepCrawlConnection(api_id, api_key, sleep=0.2)

# Guess at the name of the project
project_search_string = "PROJECT_NAME"

# Get all projects in account and print out ones that contain project_search_string set above
projects = dc.get_projects(account_id)
[(project.to_dict['name'], project.id) for project in projects if project_search_string in str(project.to_dict['name'])]

In [ ]:
project_id = "PROJECT_ID" # Update with project_id from project identified in previous step

# Get project
project = dc.get_project(account_id, project_id) # Update with account_id

# Get crawls
crawls = project.get_crawls()

# Get latest crawl (update to get previous ones)
crawl = crawls[0]

# Get all reports
all_reports = crawl.get_reports()

# Filter for reports that have rows and are flagged as an 'issue' and sort by severity
reports = []
[reports.append(report) for report in all_reports if report.to_dict['report_type'] in "basic" if report.to_dict['total_weight'] < 0]
reports = sorted(reports, key=lambda x: x.to_dict['total_weight']) 

In [ ]:
!pip install xlsxwriter
!pip install openpyxl
import xlsxwriter
from openpyxl import load_workbook
from openpyxl.styles import Alignment
from IPython.display import clear_output
clear_output()

# Name of output file
excel_filename = "OUTPUT_FILENAME.xlsx"

writer = pd.ExcelWriter(excel_filename, engine = 'openpyxl')

# Get report rows, create dataframe and export to tabs of excel file
for counter, report in enumerate(reports):
  rinfo = report.to_dict
  report_name = rinfo['report_template']
  print(f'Building report {counter}/{len(reports)} - {report_name}')
  rows = report.get_report_rows()
  report_data = []
  [report_data.append(row.data) for row in rows]
  df = pd.DataFrame.from_dict(report_data)
  if 'url' in df.columns:
    df = df.sort_values('url')
  with writer:
    df.to_excel(writer, sheet_name=report_name, index=False)
    ws = writer.sheets[report_name]
    print('Writing to workbook')
    
    # Update header rows styling
    for cell in ws[1]:
      cell.style = 'Normal'
      cell.alignment = Alignment(horizontal="left", vertical="center")
  
  clear_output()

print(f'✅ {excel_filename} created and exported')